# Models, Prompts and Parsers

In [145]:
import os
os.environ["NVIDIA_API_KEY"] = "XXXXXXXXXXXXXXXXXXXXXXXXXXXXXX"

## Direct API Calls

### NVIDIA Implementation
Let's start with direct API calls to NVIDIA ~~OpenAI~~

In [2]:
import requests, base64, os

def get_completion(prompt):
    invoke_url = "https://ai.api.nvidia.com/v1/vlm/microsoft/phi-3-vision-128k-instruct"
    
    headers = {
      "Authorization": f"Bearer {os.environ.get('NVIDIA_API_KEY')}",
      "Accept": "application/json"
    }
    
    payload = {
      "messages": [
        {
          "role": "user",
          "content": f'{prompt}'
        }
      ],
      "max_tokens": 512,
      "temperature": 1.00,
      "top_p": 0.70
    }
    response = requests.post(invoke_url, headers=headers, json=payload)
    return response.json()['choices'][0]['message']['content']


# response = get_completion("What is 1+1?").json()
# print(response['choices'][0]['message']['content'])
response = get_completion("What is 1+1?")
print(response)

1+1 is equal to 2. This is a basic arithmetic operation known as addition, where 1 is added to another 1 to produce a sum of 2.


### Ollama Implementation

In [3]:
invoke_url = "http://localhost:11434/api/generate"

def get_completion_local(prompt):
    headers = {
        "Accept": "application/json"
    }
    
    payload = {
      "model": "llama3",
      "prompt":f"{prompt}",
      "stream": False
    }
    
    response = requests.post(invoke_url, json=payload)
    return response.json()["response"]

get_completion_local("What is 1+1?")

"That's an easy one!\n\nThe answer is: 2!"

### OpenAI Implementation: Just in Case

In [4]:
#!pip install openai

In [5]:
# import openai

# os.environ["NVIDIA_API_KEY"] = XXXXXXXXXXXXXX
# openai.api_key = os.environ['OPENAI_API_KEY']
# llm_model = "gpt-3.5-turbo"

# def get_completion(prompt, model=llm_model):
#     messages = [{"role": "user", "content": prompt}]
#     response = openai.ChatCompletion.create(
#         model=model,
#         messages=messages,
#         temperature=0, 
#     )
#     return response.choices[0].message["content"]

# get_completion("What is 1+1?")

### Langchain Abstractions

In [6]:
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse,\
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

style = """American English \
in a calm and respectful tone
"""

prompt = f"""Translate the text \
that is delimited by triple backticks 
into a style that is {style}
text: ```{customer_email}```
"""

print(prompt)

Translate the text that is delimited by triple backticks 
into a style that is American English in a calm and respectful tone

text: ```
Arrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse,the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!
```



In [7]:
print(get_completion(prompt))

Oh, I am quite frustrated that my blender lid came off and caused a mess in my kitchen. And to add insult to injury, the warranty does not cover the cost of cleaning up my kitchen. I need your help right away, matey!


In [8]:
print(get_completion_local(prompt))

Here is the translated text in a calm and respectful tone using American English:

```
Gosh, I'm really upset that my blender lid came loose and splattered smoothie all over my kitchen walls! And to make things worse, the warranty doesn't cover the cost of cleaning up the mess. I could really use your help right now.
```

Let me know if you'd like any further adjustments!


## ChatAPI LangChain

### Why Use Prompt Templates?

![Why Use Prompt Templates](data/images/why-use-prompt-templates.png)

![LangChain output parsing works with prompt templates](data/images/output-parsing-works-with-prompts.png)

In [9]:
# !pip install --upgrade langchain
# !pip install langchain-nvidia-ai-endpoints

In [10]:
template_string = """Translate the text \
that is delimited by triple backticks \
into a style that is {style}. \
text: ```{text}```
"""

In [11]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(template_string)

In [12]:
prompt_template.messages[0].prompt

PromptTemplate(input_variables=['style', 'text'], template='Translate the text that is delimited by triple backticks into a style that is {style}. text: ```{text}```\n')

In [13]:
prompt_template.messages[0].prompt.input_variables

['style', 'text']

In [14]:
customer_style = """American English \
in a calm and respectful tone
"""

In [15]:
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse, \
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

In [16]:
customer_messages = prompt_template.format_messages(
                    style=customer_style,
                    text=customer_email)

In [17]:
customer_messages

[HumanMessage(content="Translate the text that is delimited by triple backticks into a style that is American English in a calm and respectful tone\n. text: ```\nArrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse, the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!\n```\n")]

In [18]:
service_reply = """Hey there customer, \
the warranty does not cover \
cleaning expenses for your kitchen \
because it's your fault that \
you misused your blender \
by forgetting to put the lid on before \
starting the blender. \
Tough luck! See ya!
"""

In [19]:
service_style_pirate = """\
a polite tone \
that speaks in English Pirate\
"""

In [20]:
service_messages = prompt_template.format_messages(
    style=service_style_pirate,
    text=service_reply)

print(service_messages[0].content)

Translate the text that is delimited by triple backticks into a style that is a polite tone that speaks in English Pirate. text: ```Hey there customer, the warranty does not cover cleaning expenses for your kitchen because it's your fault that you misused your blender by forgetting to put the lid on before starting the blender. Tough luck! See ya!
```



### ChatNVIDIA

In [309]:
from langchain_nvidia_ai_endpoints import ChatNVIDIA

# To control the randomness and creativity of the generated
# text by an LLM, use temperature = 0.0

llm_nvidia = ChatNVIDIA(model="mistralai/mixtral-8x7b-instruct-v0.1")

In [22]:
customer_response = llm_nvidia.invoke(customer_messages)

In [23]:
print(customer_response.content)

 I'm really upset that the lid of my blender came off and splattered my kitchen walls with smoothie! To make matters worse, the warranty does not cover the cost of cleaning up my kitchen. I need your help right now, friend!


In [24]:
service_response = llm_nvidia(service_messages)
print(service_response.content)

/home/mojo/miniconda3/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


 Ahoy there, me hearty customer! I'm afraid I've got some unfortunate news for ye. The warranty on yer kitchen appliance, specifically the blender, does not cover the costs of cleanin' up after a mishap. It seems ye may have forgotten to put the lid on before startin' the blender, which led to the mess. I'm truly sorry, but them's the rules, I'm afraid. We can't be coverin' every little mishap, as much as we'd like to help. I hope ye understand and have a smoother sailin' in the future. Fare thee well, me friend!


### ChatOllama

In [25]:
from langchain_community.chat_models import ChatOllama

llm_ollama = ChatOllama(model="llama3")

In [26]:
customer_response = llm_ollama(customer_messages)
print(customer_response.content)

Here's the translation:

```
Ugh, I'm absolutely livid that my blender lid flew off and splattered smoothie all over my kitchen walls! To make things worse, the warranty doesn't cover the cost of cleaning up the mess. I really need some help right now - can you lend a hand? 
```

I've kept the tone calm and respectful, while still conveying your frustration with the situation. I've also used American English spellings (e.g., "absolutely" instead of "be", "livid" instead of "fuming", etc.) to ensure the translation is suitable for an American audience.


In [27]:
service_response = llm_ollama(service_messages)
print(service_response.content)

Arrrr, greetings me hearty customer!

I be afraid that yer warranty don't cover them fancy cleanin' expenses fer yer galley (that's kitchen speak, matey). Seemingly, ye got yerself into a spot o' trouble by forgettin' to put the lid on yer trusty blender afore startin' it up. Shiver me timbers! A bit o' common sense be in order here, don't ye think?

Well, matey, it seems like ye've got yerself a case o' "Tough Luck" comin' atcha! But never fear, there be more where that came from! Just keep in mind that next time ye set sail with yer blender, make sure to have the lid securely fastened, or ye might just find yerself walkin' the plank!

Fair winds and following seas, me hearty! May yer kitchen be as smooth as a calm sea on a sunny day!


### ChatOpenAI

In [28]:
# from langchain.chat_models import ChatOpenAI
# llm_openai = ChatOpenAI(temperature=0.0, model=llm_model)

## Output Parsers

How we would like the LLM output to look like:

In [29]:
{
  "gift": False,
  "delivery_days": 5,
  "price_value": "pretty affordable!"
}

{'gift': False, 'delivery_days': 5, 'price_value': 'pretty affordable!'}

In [30]:
customer_review = """\
This leaf blower is pretty amazing.  It has four settings:\
candle blower, gentle breeze, windy city, and tornado. \
It arrived in two days, just in time for my wife's \
anniversary present. \
I think my wife liked it so much she was speechless. \
So far I've been the only one using it, and I've been \
using it every other morning to clear the leaves on our lawn. \
It's slightly more expensive than the other leaf blowers \
out there, but I think it's worth it for the extra features.
"""

review_template = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product \
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

Format the output as JSON with the following keys:
gift
delivery_days
price_value

text: {text}
"""

In [31]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(review_template)
print(prompt_template)

input_variables=['text'] messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], template='For the following text, extract the following information:\n\ngift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.\n\ndelivery_days: How many days did it take for the product to arrive? If this information is not found, output -1.\n\nprice_value: Extract any sentences about the value or price,and output them as a comma separated Python list.\n\nFormat the output as JSON with the following keys:\ngift\ndelivery_days\nprice_value\n\ntext: {text}\n'))]


In [32]:
messages = prompt_template.format_messages(text=customer_review)

In [33]:
response = llm_nvidia(messages)
print(response.content)

 {
"gift": true,
"delivery_days": 2,
"price\_value": "It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."
}


In [34]:
response = llm_ollama(messages)
print(response.content)

Here is the extracted information in JSON format:

```
{
    "gift": True,
    "delivery_days": 2,
    "price_value": ["It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."]
}
```

Note: The `price_value` key has a list value because there is only one sentence that provides information about the price or value of the product.


In [35]:
# response = llm_openai(messages)

In [36]:
type(response.content)

str

#### Parse the LLM output string into a Python dictionary

In [37]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

In [38]:
gift_schema = ResponseSchema(name="gift",
                             description="Was the item purchased\
                             as a gift for someone else? \
                             Answer 'True' if yes,\
                             'False' if not or unknown. Answer should be within quotes")
delivery_days_schema = ResponseSchema(name="delivery_days",
                                      description="How many days\
                                      did it take for the product\
                                      to arrive? If this \
                                      information is not found,\
                                      output -1.")
price_value_schema = ResponseSchema(name="price_value",
                                    description="Extract any\
                                    sentences about the value or \
                                    price, and output them as a \
                                    comma separated Python list.")

response_schemas = [gift_schema, 
                    delivery_days_schema,
                    price_value_schema]

In [39]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [40]:
format_instructions = output_parser.get_format_instructions()

In [41]:
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"gift": string  // Was the item purchased                             as a gift for someone else?                              Answer 'True' if yes,                             'False' if not or unknown. Answer should be within quotes
	"delivery_days": string  // How many days                                      did it take for the product                                      to arrive? If this                                       information is not found,                                      output -1.
	"price_value": string  // Extract any                                    sentences about the value or                                     price, and output them as a                                     comma separated Python list.
}
```


In [42]:
review_template_2 = """\
For the following text, extract the following information. 
Don't respond to me back.
Just give me the information.

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product\
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

text: {text}

{format_instructions}
"""

prompt = ChatPromptTemplate.from_template(template=review_template_2)

messages = prompt.format_messages(text=customer_review, format_instructions=format_instructions)

In [43]:
print(messages)

[HumanMessage(content='For the following text, extract the following information. \nDon\'t respond to me back.\nJust give me the information.\n\ngift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.\n\ndelivery_days: How many days did it take for the productto arrive? If this information is not found, output -1.\n\nprice_value: Extract any sentences about the value or price,and output them as a comma separated Python list.\n\ntext: This leaf blower is pretty amazing.  It has four settings:candle blower, gentle breeze, windy city, and tornado. It arrived in two days, just in time for my wife\'s anniversary present. I think my wife liked it so much she was speechless. So far I\'ve been the only one using it, and I\'ve been using it every other morning to clear the leaves on our lawn. It\'s slightly more expensive than the other leaf blowers out there, but I think it\'s worth it for the extra features.\n\n\nThe output should be a markdown co

In [44]:
response = llm_nvidia(messages)
print(response.content)

 ```json
{
	"gift": "False",
	"delivery_days": "2",
	"price_value": "It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."
}
```


In [45]:
output_dict = output_parser.parse(response.content)
print(output_dict)

{'gift': 'False', 'delivery_days': '2', 'price_value': "It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."}


In [46]:
response = llm_ollama(messages)
print(response.content)

```json
{
    "gift": "True",
    "delivery_days": "2",
    "price_value": ["It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."]
}
```


In [47]:
output_dict = output_parser.parse(response.content)

In [48]:
output_dict['price_value']

["It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."]